In [2]:
#Evaluation_only_querry_ex.txt
folder = "C:\\!DEV\\C++\\Diplom\\TemporalSummarization\\saved\\"
evaluation_1_filename = folder + "evaluation_notemp_noimp_07_12_17.txt"
evaluation_2_filename = folder + "evaluation_allin_07_12_17.txt"


In [21]:
import re
def get_evaluation_by_tag(text, tag):
    text = re.sub(r"\n", "", text)
    stories = re.findall(r"<story(.*?)>(.*?)</story>", text)
    #print(stories)
    values = []
    for story in stories:
        pattern = r"<" + re.escape(tag) + r">(.*?)</" + re.escape(tag) + r">"
        tag_data = re.search(pattern, story[1])
        values.append(float(tag_data.group(1)))
            
    return values
        
metrics_names = ["R1", "P1", "F1", "Psent1", "R2", "P2", "F2"]

eva_1 = open(evaluation_1_filename).read()
eva_2 = open(evaluation_2_filename).read()

metrics_eva_1 = []
metrics_eva_2 = []
for metric_name in metrics_names:
    metrics_eva_1.append(get_evaluation_by_tag(eva_1, metric_name))
    metrics_eva_2.append(get_evaluation_by_tag(eva_2, metric_name))


In [57]:
#simulate data
import numpy

def gen_not_real_data(metric, size):
    mean = numpy.mean(metric)
    std = numpy.std(metric)
    
    return numpy.random.normal(mean, std, size).tolist()
    

metrics_size = len(metrics_eva_1)
#for i in range(0, metrics_size):
    #metrics_eva_1[i] += gen_not_real_data(metrics_eva_1[i], 15)
    #metrics_eva_2[i] += gen_not_real_data(metrics_eva_2[i], 15)

In [23]:
import random

def abs_diff(list1, list2, size):
    return abs((sum(list1) / size) - (sum(list2) / size))

def rand_labeling(metric_1, metric_2):
    labeled_1 = []
    labeled_2 = []
    
    for i in range(0, len(metric_1)):
        if random.random() > 0.5:
            labeled_1.append(metric_1[i])
            labeled_2.append(metric_2[i])
        else:
            labeled_2.append(metric_1[i])
            labeled_1.append(metric_2[i])
    
    return [labeled_1, labeled_2]

def calc_p_value(metric_eva_1, metric_eva_2, num_of_tests):
    if len(metric_eva_1) != len(metric_eva_2):
        return -1
    size = len(metric_eva_1)
    if size < 1 or num_of_tests < 1:
        return -1
    
    diff = abs_diff(metric_eva_1, metric_eva_2, size)
    concat_metrics = metric_eva_1 + metric_eva_2
    count = 0;
    for i in range(0, num_of_tests):
        #random.shuffle(concat_metrics)
        labels = rand_labeling(metric_eva_1, metric_eva_2)
        label_1 = labels[0]
        label_2 = labels[1]
        cur_diff = abs_diff(label_1, label_2, size)
        if cur_diff > diff:
            count += 1
    
    return count / num_of_tests

metrics_size = len(metrics_eva_1)
for i in range(0, metrics_size):
    neg = 0
    for j in range(0, len(metrics_eva_1[i])):
        if metrics_eva_1[i][j] > metrics_eva_2[i][j]:
            neg += 1
    print("neg = " + str(neg) + " "+  str(len(metrics_eva_1[i])))
    print(calc_p_value(metrics_eva_1[i], metrics_eva_2[i], 5000))
    

neg = 8 15
0.9354
neg = 7 15
0.5488
neg = 7 15
0.6638
neg = 6 15
0.2328
neg = 5 15
0.6424
neg = 7 15
0.8134
neg = 7 15
0.9964
